### Feature Selection

On this part we are going to try to reduce the complexity and the correlation of our dataset.

In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn

In [7]:
data_train = pd.read_csv("transformed_train.csv")
data_test = pd.read_csv("transformed_test.csv")

In [8]:
data_train

,Survived,Age,SibSp,Parch,Fare,Sex_male,Pclass_1.0,Pclass_2.0,Pclass_3.0,Embarked_C,Embarked_Q,Embarked_S
0,0.0,0.566474,0.000,0.000000,0.055628,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.283740,0.000,0.000000,0.025374,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.396833,0.000,0.000000,0.015469,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.321438,0.125,0.000000,0.015330,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.070118,0.500,0.333333,0.061045,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
707,1.0,0.258608,0.000,0.000000,0.014932,0.0,0.0,0.0,1.0,0.0,0.0,1.0
708,0.0,0.484795,0.000,0.000000,0.060508,1.0,1.0,0.0,0.0,0.0,0.0,1.0
709,0.0,0.509927,0.250,0.000000,0.027538,1.0,0.0,0.0,1.0,0.0,0.0,1.0
710,1.0,0.170646,0.125,0.333333,0.234224,0.0,1.0,0.0,0.0,0.0,0.0,1.0


On this point we have two alternative ways of doing feature selection.

* Selecting among the columns we have
* Dimensionality reduction techniques

On this notebook I am going to follow the first aproach.

For this, we are going to employ the minimun redundancy maximun relevance (with the mutual information criterion as relevance and the correlation as redundancy) method.

In [9]:
from sklearn.feature_selection import mutual_info_regression

# Precomputing the relevances, and the redundancy
relevance = mutual_info_regression(data_train.drop("Survived", axis=1), data_train.Survived)
redundancy = np.abs(np.cov(data_train.drop("Survived", axis=1)))

m = len(relevance)

# List of features selected with the first already selected
features = [np.argmax(relevance)]
MRmr_score = [np.max(relevance)]

candidates = np.arange(m)
selected = np.array([])

# adding the first feature
f = np.argmax(relevance)
selected = np.hstack([candidates[f]])
candidates = np.delete(candidates, f)



for i in range(m-1):
    rel = relevance[candidates]
    
    #Average of the covariation coefficient of the current candidates and the selected features
    red = np.sum(redundancy[np.ix_(selected, candidates)], axis=0)
    
    mrmr = rel-red
    
    # Select the next feature
    f = np.argmax(mrmr)
    
    #Update the candidates and selected features
    features.append(candidates[f])
    MRmr_score.append(mrmr[f])
    selected = np.hstack([selected, candidates[f]])
    candidates = np.delete(candidates, f)


In [10]:
MRmr_features = pd.Series(MRmr_score, index=data_train.drop("Survived", axis=1).columns[features]) 
MRmr_features

Sex_male      0.127039
Age           0.022676
Embarked_S   -0.044543
Pclass_3.0   -0.109239
Fare         -0.175583
Pclass_1.0   -0.346850
Parch        -0.431950
Embarked_Q   -0.571950
SibSp        -0.686125
Pclass_2.0   -0.894244
Embarked_C   -1.021141
dtype: float64

We can appreciate that this more of less confirms what we have seen on the exploratory part. The most critical variable is the sex. Then the age. Then the class.

#### Comments
Given that we do not have that many features, it is not that critical to remove data features. We are going to remove them if that increases the performance. But for now they keep the same and use these results as reference.

Also we must comment that there exist a lot of techniques which do feature selection. Most of the time they do a simmilar ranking. However there is not clear evidance that one performs better than others.

We are going to define a function which mesures the score of the model given a set of features for training.

As model we are going to use a Random forest classifier since it is one of the most robust models we can use. 
In addition we are going to perform a gridsearch in order to tune the models.

The score will be the best accuracy gotten from the gridsearch with the given set of features

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

def score_model(data_train, features):
    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
    gs = GridSearchCV(model, param_grid={'max_depth': [1,2,3,4,5,6,7,8,9,10], 'n_estimators':range(10,150,20)}, scoring='roc_auc', cv=5)
    gs.fit(data_train[features], data_train["Survived"])
    return gs.best_score_

We are going to perform a recursive feature elimination.

In [27]:
# inverse recursive feature elimination
def inverse_rfe(data_train):
    features = data_train.columns.values
    features = np.delete(features, np.where(features == "Survived"))
    best_score = score_model(data_train, features)
    best_features = features

    updated = True
    while len(features) > 1 and updated:
        updated = False
        for feature in features:
            features_ = features.copy()
            features_ = np.delete(features_, np.where(features_ == feature))
            score = score_model(data_train, features_)
            if score > best_score:
                best_score = score
                best_features = features_
                updated = True

        print(f"Current best score is {best_score} \n Current selected features are {best_features}")
        features = best_features

    return best_features

inverse_rfe(data_train)


Current best score is 0.8652297656802638 
 Current selected features are ['Age' 'SibSp' 'Parch' 'Fare' 'Sex_male' 'Pclass_2.0' 'Pclass_3.0'
 'Embarked_C' 'Embarked_Q' 'Embarked_S']
Current best score is 0.8652297656802638 
 Current selected features are ['Age' 'SibSp' 'Parch' 'Fare' 'Sex_male' 'Pclass_2.0' 'Pclass_3.0'
 'Embarked_C' 'Embarked_Q' 'Embarked_S']


array(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Pclass_2.0',
       'Pclass_3.0', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype=object)

### Conclusions
At the end the features we are going tune are:


This gives us an accuracy of more or less xxx percent

Finally, we are going to do a more intense gridsearch. And we are going to create our pipeline. That way we can automatize all the steps we have done through this notebook.